# 마이리뷰
a_isbn을 사용하여 마이리뷰 수집
- 사용파일: A_information_etc_utf8.csv
- 생성파일: A_my_reviews_utf8.csv

In [ ]:
#import
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
import csv
from selenium.webdriver.common.by import By
import csv
import time
import pandas as pd

In [16]:
# CSV 파일 경로
# 알라딘-총평점-v1.0 작업 이후 A_information_etc 파일로 작업필요
file_path = 'A_information_etc_utf8.csv'

# pandas를 사용하여 CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 'a_isbn' 열의 값들을 중복 없이 리스트로 변환
a_isbn_list = df['a_isbn'].unique().tolist()

# 'item_id'에서 중복된 행만 필터링
duplicated_rows = df[df.duplicated('a_isbn', keep=False)]

# 결과 출력
if len(duplicated_rows) >= 1:
   print(duplicated_rows.sort_values(by='a_isbn'))
else:
    print('중복된 행이 없습니다')

       item_id a_isbn  total_rating
660  309951192    NaN           NaN
823   56868780    NaN           9.6


In [24]:
len(a_isbn_list)

1022

In [25]:
reviews = []
reviews_html_list = []

In [26]:
#MyReview-알라딘 내부 API 사용 

# 요청 헤더
MyReview_headers = {
    "Accept": "text/html, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Cookie": "CheckSameSite4=IsValidSameSiteSet; AladdinUser=UID=1238288727&SID=MZhgpmxTJ5jmKnv24AFjAg%3d%3d; AladdinUS=uz%2bg8Meq6f3tucYt2eBk%2fw%3d%3d&USA=0; _gcl_au=1.1.80453147.1710319532; _BS_GUUID=3jguseZr9W4O9focZdu9DZBb6ksnLWiPfyjHNi8W; _ga=GA1.1.955527986.1710319532; _fwb=191BLSAqxrjXG5K4lL8oJvI.1710319532043; _TRK_AUIDA_13987=4dd981c7ce0f287249c7f241078780ea:3; _TRK_ASID_13987=0f22e9830a291d6f37bd223e3d809829; _gcl_aw=GCL.1710380746.Cj0KCQjwncWvBhD_ARIsAEb2HW-2yZmyXOBJ6v__Us_5bPB4TuKw6AbwFUUqqoYHNCsME4FQLGoWATEaAg4qEALw_wcB; refererURL=https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168; wcs_bt=s_1c519d64863a:1710383357; _ga_VKYKBC0ZHH=GS1.1.1710378105.3.1.1710384474.60.0.0",
    "Host": "www.aladin.co.kr",
    "Referer": "https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=309295168",
    "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

#최대 200개 출력
PageSize = 200

#진척도 확인용
count=1
for aladinISBN in a_isbn_list:
    review = {
                'a_isbn': None,
                'paper_id': None,
                'review_rating': None,
                'review_like_count': None,
                'created_at': None,
                'review_comment': None,
                'reviewer' : None
            }
    try:
        # 요청 보내기
        url = f"https://www.aladin.co.kr/shop/product/getcontents.aspx?name=MyReview&ISBN={aladinISBN}&Page=1&PageSize={PageSize}&IsOrderer=1&Sort=2"
        MyReview_response = requests.get(url, headers=MyReview_headers)
        
        # 응답 확인
        # print(MyReview_response.status_code, count)  # 상태 코드 확인
      
        MyReview_soup = BeautifulSoup(MyReview_response.text, 'html.parser')
        reviews_html = {
            'aladinISBN': aladinISBN,
            'html': MyReview_soup
        }
        reviews_html_list.append(reviews_html)
        
        #리뷰 리스트로 만들기
        all_hundred_list_divs = MyReview_soup.find_all('div', class_='hundred_list')
        
        for hundred_list_div in all_hundred_list_divs:
            try:
                myreviewId = score = agree_number = createAt = MyReviewdetail = None
                ## 스코어계
                #hl_star_div_list초기화
                specific_images=[]
                # "HL_star" 클래스를 가진 div 태그 찾기
                hl_star_div = hundred_list_div.find('div', class_='HL_star')
                specific_images = hl_star_div.find_all('img', src="//image.aladin.co.kr/img/shop/2018/icon_star_on.png")
                # 특정 이미지의 갯수 계산
                score = len(specific_images)
            
                #공감수, 작성일 추출
                spans = hundred_list_div.find_all('span', class_='Ere_sub_gray8 Ere_fs13 Ere_PR10')
                agree_number = 0
                createAt = ""
                for span in spans:
                    if "공감(" in span.text:
                        agree_number = int(span.text.split('(')[1].split(')')[0])
                    elif  "-" in span.text:
                        createAt = span.text
                    else:
                        agree_number = 0
            
                #ID추출
                myreviewId_element = hundred_list_div.find('div', id=lambda x: x and x.startswith('paperShort_'))
                if myreviewId_element:
                    myreviewId = myreviewId_element['id'].split('_')[1]
                else:
                    myreviewId = None
            
                #디테일api로 접근 myreviewId필요
                detail_url = f'https://www.aladin.co.kr/ucl/shop/product/ajax/viewmypaperall_v2.aspx?paperid={myreviewId}&IsMore=1&communityType=MyReview'
                #디테일 정보 얻기
                MyReviewdetail_response = requests.get(detail_url)
                #디테일 정보 html 중 텍스트만 추출
                MyReviewdetail = BeautifulSoup(MyReviewdetail_response.text, 'html.parser').text
                reviewer = hundred_list_div.find('a', class_='Ere_sub_gray8 Ere_fs13 Ere_PR10').text
    
                
                # 딕셔너리에 정보 저장
                review = {
                    'a_isbn': aladinISBN,
                    'paper_id': myreviewId,
                    'review_rating': score,
                    'review_like_count': agree_number,
                    'created_at': createAt,
                    'review_comment': MyReviewdetail,
                    'reviewer' : reviewer
                }
            
                # 딕셔너리를 리스트에 추가
                reviews.append(review)
            except Exception as e:
                review = {
                    'a_isbn': aladinISBN,
                    'paper_id': myreviewId,
                    'review_rating': score,
                    'review_like_count': agree_number,
                    'created_at': createAt,
                    'review_comment': MyReviewdetail,
                    'reviewer' : reviewer
                    }
                reviews.append(review)
                continue
    except Exception as e:
        print(f'Unhandled exception for item_id {item_id}: {e}')
        continue  # 다음 item_id로 넘어갑니다
    print(count, time.strftime('%Y.%m.%d - %H:%M:%S'), myreviewId, aladinISBN)
    count += 1


1 2024.03.18 - 01:47:23 14085282 K752730888
2 2024.03.18 - 01:47:29 7748068 8937837706
3 2024.03.18 - 01:47:30 14370650 K162532536
4 2024.03.18 - 01:47:37 9676379 8972976199
5 2024.03.18 - 01:47:39 13363232 K952835625
6 2024.03.18 - 01:47:47 11994896 8934992492
7 2024.03.18 - 01:47:51 7338456 8985961853
8 2024.03.18 - 01:47:55 13839996 K762835176
9 2024.03.18 - 01:47:56 15292142 K392830163
10 2024.03.18 - 01:48:02 13683964 K132835758
11 2024.03.18 - 01:48:03 15381629 K232830692
12 2024.03.18 - 01:48:03 15381629 896542108X
13 2024.03.18 - 01:48:08 8656189 6000836020
14 2024.03.18 - 01:48:15 7846433 8936472690
15 2024.03.18 - 01:48:17 7010972 8965420377
16 2024.03.18 - 01:48:19 6469294 8958389575
17 2024.03.18 - 01:48:20 13693388 K702837352
18 2024.03.18 - 01:48:22 12505416 K432736724
19 2024.03.18 - 01:48:24 11787384 K032637693
20 2024.03.18 - 01:48:26 14578199 K612831580
21 2024.03.18 - 01:48:29 12764665 K152730414
22 2024.03.18 - 01:48:29 15178355 K962831052
23 2024.03.18 - 01:48:34 1

In [42]:
# 1차 저장
# 'encoding' 인자를 'utf-8'로 설정
with open('A_my_reviews_all_dupl_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(reviews)

In [34]:
#누락된 item_id있는지 확인
missing_a_isbn = set(a_isbn_list) - set([info['a_isbn'] for info in reviews])

if missing_a_isbn:
    print("Missing a_isbns:")
    for a_isbn in missing_a_isbn:
        print(a_isbn)
else:
    print('누락된 아이디 없음')

#확인필요

Missing a_isbns:
K402534471
K122636592
K312838181
K762837268
897929347X
K672633807
K042836669
8965425077
8901259303
K212730984
8917237487
8965425050
K332636376
K072636275
K502731589
K192739134
K622631613
nan
8950987740
K412534471
K902830596
K702836737
K642636979
8965422736
1156171059
K722832108
8965420954
8950995905
K382839443
8917235948
K432839784
8934944544
896542108X
K272635138
K962834900


In [35]:
#중복확인
df = pd.DataFrame(reviews)


df_clean1 = df.drop_duplicates(subset="paper_id", keep="first", ignore_index=True)
df_clean2 = df.drop_duplicates(subset="review_comment", keep="first", ignore_index=True)
df_clean3 = df.drop_duplicates(subset=["paper_id","review_comment"], keep="first", ignore_index=True)

print(len(df), len(df_clean1), len(df_clean2), len(df_clean3))

12650 12415 12321 12415


In [40]:
#중복된 행 확인
duplicated_rows = df[df.duplicated('paper_id', keep=False)]
print(duplicated_rows.sort_values(by='review_comment'))
duplicated_rows1 = duplicated_rows.sort_values(by='paper_id')
duplicated_rows1.to_csv("중복확인용2.csv")
#같은 책 다른 버전같음

          a_isbn  paper_id  review_rating  review_like_count  created_at  \
7077  K092835920  13488034              5                 50  2022-04-05   
8831  K762839932  13488034              5                 50  2022-04-05   
1223  8936434411   9486705              5                  0  2017-07-26   
9849  8936434128   9486705              5                  0  2017-07-26   
9756  8936434128   7359823              5                 13  2015-02-01   
...          ...       ...            ...                ...         ...   
9785  8936434128   7377105              5                  3  2015-02-12   
8855  K762839932  13410302              3                  5  2022-03-10   
7104  K092835920  13410302              3                  5  2022-03-10   
9811  8936434128   7296423              5                  1  2014-12-23   
1184  8936434411   7296423              5                  1  2014-12-23   

                                         review_comment    reviewer  
7077  \n\n*스포일러가 

In [41]:
#중복된 행 확인
duplicated_rows = df_clean1[df_clean1.duplicated('review_comment', keep=False)]
print(duplicated_rows.sort_values(by='review_comment'))
duplicated_rows1 = duplicated_rows.sort_values(by='review_comment')
duplicated_rows1.to_csv("중복확인용.csv")

           a_isbn  paper_id  review_rating  review_like_count  created_at  \
12389  8965707692  11257819              5                  0  2019-11-05   
11421  K622636666  11168071              4                  1  2019-10-07   
11417  K622636666  11176811              4                  2  2019-10-10   
4985   8954619576   7551134              5                  0  2015-05-19   
5136   8954637752   8173244              4                  2  2016-01-22   
...           ...       ...            ...                ...         ...   
7565   K042637691  11533981              5                  0  2020-02-27   
5971   8998046687  10551448              5                  0  2018-12-20   
3755   8926396673   8121767              5                  1  2016-01-04   
4454   8954635601   8121770              5                  1  2016-01-04   
9082   K592839643  15142573              5                  0  2023-12-16   

      review_comment      reviewer  
12389           - 접기         달콤카페   
1

In [43]:
#확인 후 저장
df_clean1.to_csv('A_my_reviews_utf8.csv', index=False, encoding='utf-8')

In [39]:
#csv로 저장
fields = list(reviews_html_list[0].keys())

# 'encoding' 인자를 'utf-8'로 설정
with open('reviews_html_list.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fields)
    writer.writeheader()
    writer.writerows(reviews_html_list)

12650